## Goal: Basic Vizualizations
- Note: will add more after have better idea of predictors. Right now I just want to tackle priors.
    - Then could go further in, look at conditions within a state

### Urban Rural Divide

In [42]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_rows", None, "display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

In [43]:
#read in data
All2019 = pd.read_csv("General_Clean_2019")
Accident2019 = pd.read_csv("2019accident.csv")

In [64]:
#subset
Urban_Rural_1 = Accident2019[["FATALS", "CITYNAME", "RUR_URBNAME", "COUNTYNAME"]]
Urban_Rural_1.head()

,FATALS,CITYNAME,RUR_URBNAME,COUNTYNAME
0,1,OPELIKA,Urban,LEE (81)
1,1,GADSDEN,Urban,ETOWAH (55)
2,1,NOT APPLICABLE,Rural,CLEBURNE (29)
3,1,RAINBOW CITY,Rural,ETOWAH (55)
4,1,NOT APPLICABLE,Urban,BALDWIN (3)


In [58]:
#Total Traffic Accidents by Urban/Rural Plot


In [4]:
#Total Traffic Fatalities

In [5]:
#Consider Adjusting for 2019 Population, etc

In [71]:
#Fatalities by City (bar)
City_Fatals = Urban_Rural_1.groupby("CITYNAME").sum()
City_Fatals.head()

,FATALS
CITYNAME,
ABBEVILLE,1
ABERDEEN,8
ABILENE,9
ABINGDON,2
ABSECON,1


In [72]:
#Fatalities by County (bar)
County_Fatals = Urban_Rural_1.groupby("COUNTYNAME").sum()
County_Fatals.head()

,FATALS
COUNTYNAME,
ABBEVILLE (1),1
ACADIA (1),14
ACCOMACK (1),5
ADA (1),29
ADAIR (1),12


### By Time

###### Month

###### Day in Month

###### Day in Week

###### Time/Hour of Day

### Weather

### Road Type